In [1]:
import open3d as o3d

def save_point_cloud(point_cloud, file_path):
    """
    Saves an Open3D point cloud to the specified file path.

    Args:
        point_cloud (open3d.geometry.PointCloud): Open3D point cloud object.
        file_path (str): File path to save the point cloud.
    """
    o3d.io.write_point_cloud(file_path, point_cloud)

# Example usage
point_cloud = o3d.geometry.PointCloud()  # Assume you have an Open3D point cloud object
file_path = 'cam1_pc1.ply'    # Output file path

save_point_cloud(point_cloud, file_path)


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D WARNING] Write PLY failed: point cloud has 0 points.


In [16]:
pc1 = o3d.io.read_point_cloud('cam1_pc1.ply')
pc2 = o3d.io.read_point_cloud('cam1_pc2.ply')
cam0_pc1 = o3d.io.read_point_cloud('cam0_pc1.ply')

In [6]:
o3d.visualization.draw_geometries([pc1, pc2])

In [57]:
import open3d as o3d

def remove_small_pieces(point_cloud, eps, min_samples, density_check=False):
    """
    Removes small scattered pieces from an Open3D point cloud using DBSCAN clustering.

    Args:
        point_cloud (open3d.geometry.PointCloud): Open3D point cloud object.
        eps (float): The maximum distance between two samples for them to be considered as in the same neighborhood.
        min_samples (int): The number of samples in a neighborhood for a point to be considered as a core point.

    Returns:
        open3d.geometry.PointCloud: Filtered point cloud with small pieces removed.
    """
    if density_check:
        if get_bbox_points_density(point_cloud) > 3e8:  # 3e8 is roughly the density of half a ball's surface
            return point_cloud

    labels = point_cloud.cluster_dbscan(eps=eps, min_points=min_samples)

    core_mask = np.asarray(labels) != -1  # Identify core points (not noise)
    filtered_cloud = point_cloud.select_by_index(np.where(core_mask)[0])

    return filtered_cloud

# Example usage
point_cloud = cam0_pc1  # Assume you have an Open3D point cloud object
eps = 0.02  # Maximum distance between points in the same cluster
min_samples = 1000  # Minimum number of points in a cluster

o3d.visualization.draw_geometries([point_cloud])
filtered_point_cloud = remove_small_pieces(point_cloud, eps, min_samples)
o3d.visualization.draw_geometries([filtered_point_cloud])

In [52]:
def get_bbox_points_density(pc):
    max_bound = pc.get_axis_aligned_bounding_box().get_max_bound()
    min_bound = pc.get_axis_aligned_bounding_box().get_min_bound()
    n_points = np.asarray(pc.points).shape[0]

    return n_points / np.prod(max_bound - min_bound)

labels

In [58]:
get_bbox_points_density(point_cloud)

23184060.98189734

In [59]:
get_bbox_points_density(filtered_point_cloud)

29328193.177949943

In [46]:
bound_diff = max_bound - min_bound
bound_diff

array([0.09037522, 0.16260598, 0.17401076])

In [48]:
np.prod(bound_diff) / np.asarray(filtered_point_cloud.points).shape[0]

1.525220032092304e-07

In [50]:
o3d.visualization.draw_geometries([point_cloud, point_cloud.get_axis_aligned_bounding_box()])

In [49]:
o3d.visualization.draw_geometries([filtered_point_cloud, filtered_point_cloud.get_axis_aligned_bounding_box()])
max_bound = filtered_point_cloud.get_axis_aligned_bounding_box().get_max_bound()
min_bound = filtered_point_cloud.get_axis_aligned_bounding_box().get_min_bound()
